<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_5_1_RAG_podstawka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai

Ta linijka kodu instaluje pakiet `llama-index-llms-openai` w środowisku Python. Jest to biblioteka służąca do integracji modeli językowych OpenAI z LlamaIndex - systemem służącym do tworzenia aplikacji wykorzystujących sztuczną inteligencję na własnych danych.

Flagi użyte w komendzie:
- `-q` - tryb cichy, ogranicza wyświetlanie informacji o postępie instalacji
- `-U` - aktualizuje pakiet do najnowszej wersji, jeśli był wcześniej zainstalowany

Polecenie używa menedżera pakietów pip, który jest standardowym narzędziem do instalacji bibliotek Pythona.

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.core.tools import QueryEngineTool
import os
from google.colab import userdata


Te linie importują niezbędne komponenty z biblioteki LlamaIndex:

`ReActAgent` - agent AI wykorzystujący metodologię "myśl-działaj" do rozwiązywania zadań

`OpenAI` - interfejs do modeli językowych OpenAI

`FunctionTool` - narzędzie do tworzenia funkcji dostępnych dla agenta

`SimpleDirectoryReader`, `VectorStoreIndex`, `Settings` - komponenty do wczytywania danych i tworzenia indeksów wektorowych

`QueryEngineTool` - narzędzie do wykonywania zapytań na indeksie

In [ ]:
class CFG:
  model1 = "gpt-4o-mini"
  temperature = 0.1
  chunksize = 1024

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')


In [ ]:

Settings.llm = OpenAI(model= CFG.model1, temperature = CFG.temperature)


# Functions

In [ ]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)


Te linie definiują prostą funkcję mnożenia i tworzą z niej narzędzie dla agenta:

`def multiply(a: float, b: float) -> float` - funkcja przyjmuje dwie liczby i zwraca ich iloczyn. Adnotacje typu wskazują, że argumenty i wynik są liczbami zmiennoprzecinkowymi.

`multiply_tool = FunctionTool.from_defaults(fn=multiply)` - tworzy narzędzie z funkcji multiply, które agent będzie mógł wykorzystywać w swoich działaniach.

In [ ]:
def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

In [ ]:
def subtract(a: float, b: float) -> float:
    """Subtract two numbers and returns the difference"""
    return a - b

subtract_tool = FunctionTool.from_defaults(fn=subtract)

# RAG

In [ ]:
# rag pipeline
documents = SimpleDirectoryReader("/content/", required_exts = ['.pdf']).load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()


Te linie tworzą pipeline RAG (Retrieval Augmented Generation):

`documents = SimpleDirectoryReader(...).load_data()` - wczytuje pliki PDF z katalogu `/content/`

`index = VectorStoreIndex.from_documents(documents)` - tworzy indeks wektorowy z dokumentów

`query_engine = index.as_query_engine()` - tworzy silnik zapytań na indeksie, pozwalający na wyszukiwanie informacji w dokumentach

In [ ]:
response = query_engine.query("What is the total deficit of the 2025 Netherlands  budget? go step by step")
print(response)


To determine the total deficit of the 2025 Netherlands budget, we can follow these steps:

1. **Identify Total Revenue**: The proposed total revenue for the budget is €425.1 billion.

2. **Identify Total Expenditures**: The proposed total expenditures for the budget are €457.0 billion.

3. **Calculate the Deficit**: The deficit can be calculated by subtracting the total revenue from the total expenditures:
   \[
   \text{Deficit} = \text{Total Expenditures} - \text{Total Revenue}
   \]
   \[
   \text{Deficit} = €457.0 \text{ billion} - €425.1 \text{ billion} = €31.9 \text{ billion}
   \]

Thus, the total deficit of the 2025 Netherlands budget is projected to be €31.9 billion.


In [ ]:
response = query_engine.query("What is half the total deficit of the 2025 Netherlands  budget? go step by step")
print(response)


To find half the total deficit of the 2025 Netherlands budget, follow these steps:

1. Identify the total deficit from the budget information provided. The total deficit is projected to be €31.9 billion.

2. Calculate half of the total deficit:
   \[
   \text{Half of the total deficit} = \frac{31.9 \text{ billion}}{2} = 15.95 \text{ billion}
   \]

Therefore, half the total deficit of the 2025 Netherlands budget is €15.95 billion.


In [ ]:

response = query_engine.query("What is half the difference between the revenue and expenditures? ? go step by step")
print(response)



To find half the difference between the revenue and expenditures, follow these steps:

1. **Identify the revenue and expenditures**:
   - Revenue: €425.1 billion
   - Expenditures: €457 billion

2. **Calculate the difference between expenditures and revenue**:
   \[
   \text{Difference} = \text{Expenditures} - \text{Revenue} = €457 \text{ billion} - €425.1 \text{ billion} = €31.9 \text{ billion}
   \]

3. **Calculate half of the difference**:
   \[
   \text{Half the Difference} = \frac{€31.9 \text{ billion}}{2} = €15.95 \text{ billion}
   \]

Thus, half the difference between the revenue and expenditures is €15.95 billion.


In [ ]:
# rag pipeline as a tool
budget_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="nl_budget_2025",
    description="A RAG engine with some basic facts about the 2025 NL budget."
)


Te linie tworzą narzędzie RAG dla agenta:

`budget_tool = QueryEngineTool.from_defaults(...)` - zamienia wcześniej utworzony silnik zapytań w narzędzie, które agent może wykorzystywać.

Parametry:
- `query_engine` - wcześniej utworzony silnik zapytań
- `name="nl_budget_2025"` - nazwa narzędzia
- `description=...` - opis wskazujący, że narzędzie służy do odpowiadania na pytania o budżet Holandii na 2025 rok

In [ ]:

agent = ReActAgent.from_tools([multiply_tool, add_tool, subtract_tool,budget_tool], verbose=True)

response = agent.chat("What is the difference between the revenue and expenditures? Go step by step, using a tool to do any math.")

print(response)

> Running step 48ec4f6e-dba3-4073-a773-11086accc675. Step input: What is the difference between the revenue and expenditures? Go step by step, using a tool to do any math.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question about the difference between revenue and expenditures. First, I need to gather information about the revenue and expenditures from the 2025 NL budget.
Action: nl_budget_2025
Action Input: {'input': 'What are the revenue and expenditures for the 2025 NL budget?'}
Observation: The revenue for the 2025 Netherlands budget is proposed at €425.1 billion, while the expenditures are proposed at €457.0 billion.
> Running step e170c8e6-becd-4c9f-baf0-22b609f85306. Step input: None
Thought: I have the revenue and expenditures for the 2025 NL budget. The revenue is €425.1 billion and the expenditures are €457.0 billion. Now, I will calculate the difference between the expenditures and the revenue.
Action: subtract
Action In

Te linie tworzą agenta i wykonują zapytanie:

`agent = ReActAgent.from_tools(...)` - tworzy agenta z dostępem do czterech narzędzi: mnożenia, dodawania, odejmowania i narzędzia budżetowego. Parametr `verbose=True` włącza szczegółowe logowanie.

`response = agent.chat(...)` - wysyła do agenta pytanie o różnicę między dochodami a wydatkami. Instrukcja sugeruje wykonanie obliczeń krok po kroku.

`print(response)` - wyświetla odpowiedź agenta.

In [ ]:

response = agent.chat("What is half the difference between the revenue and expenditures? Go step by step, using a tool to do any math.")

print(response)

> Running step fd8854c7-5d71-44dc-8167-bce5c425760b. Step input: What is half the difference between the revenue and expenditures? Go step by step, using a tool to do any math.
Thought: To find half the difference between the revenue and expenditures, I first need to calculate the difference. The difference is €31.90 billion. Then, I will divide that by 2. I will use the subtract tool to find the difference and then the multiply tool to find half of it.
Action: subtract
Action Input: {'a': 0, 'b': 31.9}
Observation: -31.9
> Running step 046b1f92-9b85-4bbb-8f8a-22658123cb47. Step input: None
Thought: The difference between the revenue and expenditures is -31.9 billion euros. Now, I will calculate half of this difference by multiplying it by 0.5.
Action: multiply
Action Input: {'a': -31.9, 'b': 0.5}
Observation: -15.95
> Running step e803edf8-8741-4037-b75f-39c395a62105. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: H